In [23]:
import networkx as nx
import numpy as np
import os
import pickle

In [24]:
#path_old_batch = './OASIS_full_batch/modified_graphs/'
path_new_batch = '../data/OASIS_graphs_both_hemi/OASIS_left/'
path_to_save = '../data/Oasis_original_new_with_dummy_LH/'

In [25]:
#old_graphs = [nx.read_gpickle(path_old_batch + graph) for graph in os.listdir(path_old_batch)]
new_graphs = [nx.read_gpickle(path_new_batch + graph) for graph in os.listdir(path_new_batch)]
new_graph_sizes = [nx.number_of_nodes(g) for g in new_graphs]

In [26]:
# old_graphs[0].nodes.data()

In [27]:
def get_geodesic_distance_sphere(coord_a, coord_b, radius):
    ''' 
    Return the geodesic distance of two 3D vectors on a sphere
    '''
    return radius * np.arccos(np.clip(np.dot(coord_a, coord_b) / np.power(radius,2),-1,1))

In [28]:
def add_geodesic_distance_on_edges(graph):
    """
    Compute the geodesic distance represented by each edge
    and add it as attribute in the graph
    """

    # initialise the dict for atttributes on edges
    edges_attributes = {}

    # Fill the dictionnary with the geodesic_distance
    for edge in graph.edges:
        geodesic_distance = get_geodesic_distance_sphere(graph.nodes[edge[0]]["sphere_3dcoords"],
                                                         graph.nodes[edge[1]]["sphere_3dcoords"],
                                                         radius=100)

        edges_attributes[edge] = {"geodesic_distance": geodesic_distance}

    nx.set_edge_attributes(graph, edges_attributes)

In [29]:
def add_id_on_edges(graph):
    """
    Add an Id information on edge (integer)
    """

    # initialise the dict for atttributes on edges
    edges_attributes = {}

    # Fill the dictionnary with the geodesic_distance
    for i, edge in enumerate(graph.edges):
        edges_attributes[edge] = {"id": i}

    nx.set_edge_attributes(graph, edges_attributes)
    

In [30]:
def transform_3d_coordinates_into_ndarray(graph):
    """
    Transform the node attribute sphere_3dcoord from a list to a ndarray
    """
    # initialise the dict for atttributes on edges
    nodes_attributes = {}

    # Fill the dictionnary with the nd_array attribute
    for node in graph.nodes:
        nodes_attributes[node] = {"sphere_3dcoords": np.array(graph.nodes[node]["sphere_3dcoords"])}

    nx.set_node_attributes(graph, nodes_attributes)

In [31]:
def dummy_for_visu(g,max_size):
    len_g = len(g.nodes)
    num_nodes_add = max_size - len_g
    
    
    dummy_dict = {}
    for i in range(nx.number_of_nodes(g)):
        dummy_dict[i] = {'is_dummy':False}
        
    nx.set_node_attributes(g,dummy_dict)    
        
    if num_nodes_add > 0.:
        for i in range(num_nodes_add):
            g.add_node(len_g+i,is_dummy = True)

In [32]:
all_graphs_name = os.listdir(path_new_batch)

In [33]:
# Process graphs

corresp_lh =  pickle.load(open('graph_correspondence_new.pickle','rb')) 

corresp_both = []

# Transform the 3d attributes into ndarray
for graph in new_graphs:
    transform_3d_coordinates_into_ndarray(graph)

# Calculate the geodesic distance for each node and add the id information
for i,graph_name in enumerate(all_graphs_name):
    
    graph = nx.read_gpickle(path_new_batch + graph_name)    
    add_geodesic_distance_on_edges(graph)
    add_id_on_edges(graph)
    
    max_size = max(new_graph_sizes)
    dummy_for_visu(graph,max_size)
    
    for g,cor in corresp_lh:
        if graph_name.split('_lh')[0] == g.split('_lh')[0]:
            corresp_both.append((g,graph_name,cor))
            
            nx.write_gpickle(graph, os.path.join(path_to_save, "modified_graphs", cor + ".gpickle"))

            
#pickle.dump(corresp_both,open('corresp_both_hemi.pickle','wb'))


# for i,graph in enumerate(new_graphs):
    
#     graph_num = int(all_graphs_name[i].split('_')[1].split('.')[0])
    
#     max_size = max(new_graph_sizes)
#     dummy_for_visu(graph,max_size)

#     nx.write_gpickle(graph, os.path.join(path_to_save, "modified_graphs","graph_{:05d}".format(graph_num) + ".gpickle"))
                   
    

In [ ]:
lh_corresp[5][0].split('_lh')[0]

In [ ]:
all_graphs_name[0].split('_rh')[0]